## Basic Statistics on UKB dataset

Set up global variables and import modules

In [12]:
import os
import sys
import pandas as pd
from matplotlib import pyplot as plt
from pathlib import Path 

ukb_root = '/project_freenas/3022017.02/UKB'
sys.path.append(os.path.join(ukb_root,'scripts'))
from ukb_utils import get_variables_UKB, lookup_UKB

ukb_idp_dir = os.path.join(ukb_root,'phenotypes','current')

#path for saving results
savehere = Path('/Desktop/Diffusion/diffusion_nm')

Read dataframe

In [ ]:
basic_demo = pd.read_csv(os.path.join(ukb_idp_dir,'01_basic_demographics.csv'),nrows=1)
brain_IDPs = pd.read_csv(os.path.join(ukb_idp_dir,'31_brain_IDPs.csv'),nrows=1)

Lookup field IDs with Lookup_UKB function <br>
This will to help figure out what fields are present in the dataframe

In [ ]:
def list_field_names(data_frame):
  fields_ID = data_frame.columns
  field_list = pd.DataFrame(columns=['Field', 'FieldID'])
  for idf in fields_ID[1:]:
    fld = lookup_UKB(field_ids=[idf]) 
    field_list = field_list.append(fld)
  return field_list

Save resulted dataframes for later use

In [17]:
#brain_IDPs_fields = list_field_names(brain_IDPs)
#brain_IDPs_fields.to_csv('/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/brain_IDPs_fields.csv')
#basic_demo_fields = list_field_names(basic_demo)
#basic_demo_fields.to_csv('/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/basic_demo_fields.csv')

Turns out some dataframes contain too many fields to be inspected manually.
As an alternative, specific metrics (field names) can be looked up on the UK Biobank website and their respective field codes can be used to extract data from the containing dataframe.